<a href="https://colab.research.google.com/github/savinthie/Final_Year_Project_IDP_2024-2025/blob/main/Model_2_CNN_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, ReLU, Concatenate, MaxPooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/FYP 2024 25/USDataset.csv', header=1)
df = df.fillna(0)

Mounted at /content/drive


In [ ]:
# Data Preprocessing

df.columns = [col.lower().replace(' ', '').replace('.', '') for col in df.columns]
cols_to_keep = ['stateabv', 'county', 'family', 'housing', 'food', 'transportation',
                'healthcare', 'othernecessities', 'childcare', 'taxes', 'total',
                'median_family_income', 'num_counties_in_st']
df1 = df[cols_to_keep].copy()

df1['median_family_income'] = df1['median_family_income'].replace(',', '', regex=True).astype(float)
#getting the median family income on monthly basis
df1['median_family_income'] = df1['median_family_income'].map(lambda x: x/12)

In [ ]:
# Feature Engineering
df1['n_parents'] = df1['family'].str.slice(0, 1).astype(int)
df1['n_children'] = df1['family'].str.slice(2, 3).astype(int)
df1['n_members'] = df1['n_parents'] + df1['n_children']
df1['financial_stability'] = df1['median_family_income'] / df1['total']
df1["per_member_cost"] = df1["total"] / df1["n_members"]
df1["child_expense_cost"] = df1["per_member_cost"]*df1["n_children"]
df1["parent_expense_cost"] = df1["per_member_cost"]*df1["n_parents"]
df1["other_expense_cost"] = df1["total"] - (df1["child_expense_cost"]+df1["parent_expense_cost"])
# fixing the 0 child issue
df1["zero_childcare_cost"] = df1['n_children'].map(lambda x: 0 if x < 1 else 1)

In [ ]:
# Splitting the data
X = df1[['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']+['per_member_cost','child_expense_cost','parent_expense_cost','other_expense_cost','zero_childcare_cost']].values
y_expenses = df1[['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']].values

target_col_list = ['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']

# Scaling
scaler_X = MinMaxScaler()

# Load the X scaler
# scaler_X = joblib.load('scaler_X.pkl')

X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()

# Load the y scaler
# scaler_y = joblib.load('scaler_y.pkl')
y_exp_scaled = scaler_y.fit_transform(y_expenses)


# Save the X scaler
joblib.dump(scaler_X, 'scaler_X.pkl')

# Save the y scaler
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [ ]:
# this method is used to avoid the issue in the mape and smape value for the childcare expenses since the childcare expenses consist of 0 value where the smape and mape are sensitive of.
def replace_zeros_with_ones(a, b):
    # Iterate through both lists simultaneously
    for i in range(len(a)):
        # If both value is 0, replace both with 1
        if a[i] == 0 and b[i] == 0:
            a[i] = 1
            b[i] = 1
    return a, b

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculateSmape(y_true, y_pred):
    y_true, y_pred = replace_zeros_with_ones(y_true, y_pred)
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape_value = np.mean(numerator / denominator) * 100
    return smape_value

In [ ]:
# Cross-Validation Setup
kf = KFold(n_splits=5, shuffle=True, random_state=100)

# Metrics
def print_metrics(y_true, y_pred, task_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{task_name} - MSE: {mse:.4f}, R2: {r2:.4f}")

def regr_report(x, y):
    mae = round(mean_absolute_error(x, y), 4)
    r2 = round(r2_score(x, y), 4)
    mse = round(mean_squared_error(x, y), 4)
    rmse = round(np.sqrt(mean_squared_error(x, y)), 4)
    smape_score = calculateSmape(x, y)
    return f'MAE: {mae}, R-Squared: {r2}, RMSE: {rmse}, MSE: {mse} ,smape: {smape_score}'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import xgboost as xgb

num_children_col = 3  # 4th position, 0-indexed
childcare_exp_col = target_col_list.index('childcare')  # Replace 'childcare_exp' with the actual target name

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# L2 regularization strength (you can tune this value)
l2_strength = 0.01

# Hybrid Model Definition
def create_hybrid_model(input_shape, output_shape):
    input_layer = Input(shape=input_shape)
    # CNN part
    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = Flatten()(x)
    # MLP part
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    # Multiple regression outputs
    output_layers = [Dense(1, activation='linear', name=f'target_{col}')(x) for col in target_col_list]

    model = Model(inputs=input_layer, outputs=output_layers)
    return model

In [ ]:
global X_train_global, X_test_global, X_val_global
global y_train_global, y_test_global, y_val_global

In [ ]:
# Cross-Validation with Evaluation
def cross_val_with_evaluation(X_scaled, y_exp_scaled, kf, num_epochs=200):
    global X_train_global, X_test_global, X_val_global
    global y_train_global, y_test_global, y_val_global
    fold = 1
    for train_index, val_index in kf.split(X_scaled):
        print(f"Fold {fold}/{kf.get_n_splits()}")
        X_train, X_test_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test_val = y_exp_scaled[train_index], y_exp_scaled[val_index]

        X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=100) # 50/50 split of remaining 20%

        X_train_global, X_test_global, X_val_global = X_train, X_test, X_val
        y_train_global, y_test_global, y_val_global = y_train, y_test, y_val

        # Reshaping data for CNN
        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]
        X_test = X_test[..., np.newaxis]

        model = create_hybrid_model((X_train.shape[1], 1), y_train.shape[1])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

        model.summary()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Training
        model.fit(X_train, [y_train[:, i] for i in range(y_train.shape[1])],
                  validation_data=(X_val, [y_val[:, i] for i in range(y_val.shape[1])]),
                  epochs=num_epochs, batch_size=64, verbose=1, callbacks= [early_stopping])

        # Predictions
        y_train_pred = np.column_stack(model.predict(X_train))
        y_val_pred = np.column_stack(model.predict(X_val))
        y_test_pred = np.column_stack(model.predict(X_test)) #Prediction on test set

        # Inverse scaling
        y_train_pred_original = scaler_y.inverse_transform(y_train_pred)
        y_train_original = scaler_y.inverse_transform(y_train)
        y_val_pred_original = scaler_y.inverse_transform(y_val_pred)
        y_val_original = scaler_y.inverse_transform(y_val)
        y_test_pred_original = scaler_y.inverse_transform(y_test_pred) #Inverse transform for test set
        y_test_original = scaler_y.inverse_transform(y_test) #Inverse transform for test set

        # Enforce the rule after predictions for childcare expense
        # The issue is likely caused by X_train having an extra dimension due to[..., np.newaxis].
        # We need to select the original features for the condition
        y_train_pred_original[:, childcare_exp_col] = np.where(
            X_train[:, num_children_col, 0] == 0, 0, y_train_pred_original[:, childcare_exp_col]) #Using X_train[:, num_children_col, 0] instead of X_train[:, num_children_col]

        y_val_pred_original[:, childcare_exp_col] = np.where(
            X_val[:, num_children_col, 0] == 0, 0, y_val_pred_original[:, childcare_exp_col]) #Using X_val[:, num_children_col, 0] instead of X_val[:, num_children_col]

        y_test_pred_original[:, childcare_exp_col] = np.where(
            X_test[:, num_children_col, 0] == 0, 0, y_test_pred_original[:, childcare_exp_col]) #Using X_test[:, num_children_col, 0] instead of X_test[:, num_children_col]

        # Metrics for each target
        for i, target in enumerate(target_col_list):
            print(f'Model Results for {target.capitalize()}:')
            print('Train Data:', regr_report(y_train_original[:, i], y_train_pred_original[:, i]))
            print('Validation Data:', regr_report(y_val_original[:, i], y_val_pred_original[:, i]))
            print('Test Data:', regr_report(y_test_original[:, i], y_test_pred_original[:, i])) #Print metrics for test set
            print()

        fold += 1
    return model

model_combine = cross_val_with_evaluation(X_scaled, y_exp_scaled,kf)



Fold 1/5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 11, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 11, 8)          │             32 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 88)             │              0 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         11,392 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_housing (Dense)    │ (None, 1)              │             65 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_food (Dense)       │ (None, 1)              │             65 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_transportation     │ (None, 1)              │             65 │ dense_1[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_healthcare (Dense) │ (None, 1)              │             65 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_othernecessities   │ (None, 1)              │             65 │ dense_1[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_childcare (Dense)  │ (None, 1)              │             65 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_taxes (Dense)      │ (None, 1)              │             65 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 20,135 (78.65 KB)

 Trainable params: 20,135 (78.65 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 11, 1)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 11, 8)          │             32 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)       │ (None, 88)             │              0 │ conv1d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 128)            │         11,392 │ flatten_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 64)             │          8,256 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_housing (Dense)    │ (None, 1)              │             65 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_food (Dense)       │ (None, 1)              │             65 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_transportation     │ (None, 1)              │             65 │ dense_9[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_healthcare (Dense) │ (None, 1)              │             65 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_othernecessities   │ (None, 1)              │             65 │ dense_9[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_childcare (Dense)  │ (None, 1)              │             65 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_taxes (Dense)      │ (None, 1)              │             65 │ dense_9[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 20,135 (78.65 KB)

 Trainable params: 20,135 (78.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0581 - target_childcare_loss: 0.0087 - target_food_loss: 0.0087 - target_healthcare_loss: 0.0164 - target_housing_loss: 0.0043 - target_othernecessities_loss: 0.0070 - target_taxes_loss: 0.0038 - target_transportation_loss: 0.0093 - val_loss: 0.0140 - val_target_childcare_loss: 0.0020 - val_target_food_loss: 6.9249e-04 - val_target_healthcare_loss: 0.0049 - val_target_housing_loss: 0.0013 - val_target_othernecessities_loss: 0.0011 - val_target_taxes_loss: 5.3372e-04 - val_target_transportation_loss: 0.0033
Epoch 2/200
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0139 - target_childcare_loss: 0.0021 - target_food_loss: 6.0766e-04 - target_healthcare_loss: 0.0050 - target_housing_loss: 0.0014 - target_othernecessities_loss: 9.2155e-04 - target_taxes_loss: 5.1158e-04 - target_transportation_loss: 0.0034 - val_loss: 0.0140 - val_target_childcare_loss: 0.0019 - val_target_food_loss: 7.7266e-04 - val_target_healthcare_loss

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

def cross_val_with_evaluation(X_scaled, y_exp_scaled, kf, num_epochs=200):
    fold = 1
    cnn_results, xgb_results = [], []

    for train_index, val_index in kf.split(X_scaled):
        print(f"Fold {fold}/{kf.get_n_splits()}")

        # Split data
        X_train, X_test_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test_val = y_exp_scaled[train_index], y_exp_scaled[val_index]

        X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=100)

        # CNN: Train and Predict
        print("Training CNN...")
        cnn_model = create_hybrid_model((X_train.shape[1], 1), y_train.shape[1])
        cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        cnn_model.fit(X_train[..., np.newaxis], [y_train[:, i] for i in range(y_train.shape[1])],
                      validation_data=(X_val[..., np.newaxis], [y_val[:, i] for i in range(y_val.shape[1])]),
                      epochs=num_epochs, batch_size=64, verbose=1, callbacks=[early_stopping])

        y_train_pred_cnn = np.column_stack(cnn_model.predict(X_train[..., np.newaxis]))
        y_val_pred_cnn = np.column_stack(cnn_model.predict(X_val[..., np.newaxis]))
        y_test_pred_cnn = np.column_stack(cnn_model.predict(X_test[..., np.newaxis]))

        # XGBoost: Train and Predict
        print("Training XGBoost...")
        xgb_models = []
        y_train_pred_xgb, y_val_pred_xgb, y_test_pred_xgb = [], [], []

        for i in range(y_train.shape[1]):
            dtrain = xgb.DMatrix(X_train, label=y_train[:, i])
            dval = xgb.DMatrix(X_val, label=y_val[:, i])
            dtest = xgb.DMatrix(X_test)

            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': 0.1,
                'max_depth': 6,
                'seed': 100
            }

            xgb_model = xgb.train(params, dtrain, num_boost_round=200, evals=[(dval, 'validation')],
                                  early_stopping_rounds=10, verbose_eval=False)
            xgb_models.append(xgb_model)

            y_train_pred_xgb.append(xgb_model.predict(dtrain))
            y_val_pred_xgb.append(xgb_model.predict(dval))
            y_test_pred_xgb.append(xgb_model.predict(dtest))

        y_train_pred_xgb = np.column_stack(y_train_pred_xgb)
        y_val_pred_xgb = np.column_stack(y_val_pred_xgb)
        y_test_pred_xgb = np.column_stack(y_test_pred_xgb)

        # Combine Predictions (Weighted Average)
        alpha = 0.5
        y_train_pred_combined = alpha * y_train_pred_cnn + (1 - alpha) * y_train_pred_xgb
        y_val_pred_combined = alpha * y_val_pred_cnn + (1 - alpha) * y_val_pred_xgb
        y_test_pred_combined = alpha * y_test_pred_cnn + (1 - alpha) * y_test_pred_xgb

        # Inverse scaling
        y_train_pred_original = scaler_y.inverse_transform(y_train_pred_combined)
        y_train_original = scaler_y.inverse_transform(y_train)
        y_val_pred_original = scaler_y.inverse_transform(y_val_pred_combined)
        y_val_original = scaler_y.inverse_transform(y_val)
        y_test_pred_original = scaler_y.inverse_transform(y_test_pred_combined)
        y_test_original = scaler_y.inverse_transform(y_test)

        # Metrics
        for i, target in enumerate(target_col_list):
            print(f'Results for {target.capitalize()}:')
            print(f'  Train R^2: {r2_score(y_train_original[:, i], y_train_pred_original[:, i]):.4f}')
            print(f'  Validation R^2: {r2_score(y_val_original[:, i], y_val_pred_original[:, i]):.4f}')
            print(f'  Test R^2: {r2_score(y_test_original[:, i], y_test_pred_original[:, i]):.4f}')
            print()

        fold += 1

    return cnn_model, xgb_models

cnn_model, xgb_models = cross_val_with_evaluation(X_scaled, y_exp_scaled, kf)



Fold 1/5
Training CNN...
Epoch 1/200
393/393 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.0776 - target_childcare_loss: 0.0134 - target_food_loss: 0.0048 - target_healthcare_loss: 0.0134 - target_housing_loss: 0.0095 - target_othernecessities_loss: 0.0069 - target_taxes_loss: 0.0089 - target_transportation_loss: 0.0207 - val_loss: 0.0133 - val_target_childcare_loss: 0.0021 - val_target_food_loss: 5.3239e-04 - val_target_healthcare_loss: 0.0045 - val_target_housing_loss: 0.0013 - val_target_othernecessities_loss: 8.5791e-04 - val_target_taxes_loss: 4.8541e-04 - val_target_transportation_loss: 0.0035
Epoch 2/200
393/393 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0135 - target_childcare_loss: 0.0020 - target_food_loss: 6.1540e-04 - target_healthcare_loss: 0.0049 - target_housing_loss: 0.0013 - target_othernecessities_loss: 8.7017e-04 - target_taxes_loss: 4.9101e-04 - target_transportation_loss: 0.0033 - val_loss: 0.0126 - val_target_childcare_loss: 0.0019 - val_target_food_loss: 4.8946e-04 

In [ ]:
# Cross-Validation with Evaluation for XGBoost
def cross_val_with_evaluation(X_scaled, y_exp_scaled, kf, num_boost_rounds=4000):
    fold = 1
    childcarelist = []
    childcarelist_pred = []
    for train_index, val_index in kf.split(X_scaled):
        models = []
        print(f"Fold {fold}/{kf.get_n_splits()}")
        X_train, X_test_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test_val = y_exp_scaled[train_index], y_exp_scaled[val_index]

        # Split validation and test data
        X_test, X_val, y_test, y_val = train_test_split(
            X_test_val, y_test_val, test_size=0.5, random_state=100
        )  # 50/50 split of remaining 20%

        # Train XGBoost model for each target variable
        # Store predictions for all targets in this fold
        all_y_train_pred = []
        all_y_val_pred = []
        all_y_test_pred = []

        for i in range(y_train.shape[1]):
            dtrain = xgb.DMatrix(X_train, label=y_train[:, i])
            dval = xgb.DMatrix(X_val, label=y_val[:, i])
            dtest = xgb.DMatrix(X_test)

            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': 0.01,
                'max_depth': 6,
                'n_estimators': num_boost_rounds
            }

            evals = [(dtrain, 'train'), (dval, 'eval')]
            model = xgb.train(
                params, dtrain, num_boost_rounds, evals, early_stopping_rounds=50, verbose_eval=50
            )
            models.append(model) #Not required here

            # Predictions
            y_train_pred = model.predict(dtrain)
            y_val_pred = model.predict(dval)
            y_test_pred = model.predict(dtest)

            # Append predictions for current target to the list
            all_y_train_pred.append(y_train_pred)
            all_y_val_pred.append(y_val_pred)
            all_y_test_pred.append(y_test_pred)

        # Stack predictions for all targets to get the original shape
        y_train_pred = np.column_stack(all_y_train_pred)
        y_val_pred = np.column_stack(all_y_val_pred)
        y_test_pred = np.column_stack(all_y_test_pred)

        # Inverse scaling
        y_train_pred_original = scaler_y.inverse_transform(y_train_pred) #Now y_train_pred has 7 columns matching scaler_y
        y_train_original = scaler_y.inverse_transform(y_train)
        y_val_pred_original = scaler_y.inverse_transform(y_val_pred)  #Now y_val_pred has 7 columns matching scaler_y
        y_val_original = scaler_y.inverse_transform(y_val)
        y_test_pred_original = scaler_y.inverse_transform(y_test_pred) #Now y_test_pred has 7 columns matching scaler_y
        y_test_original = scaler_y.inverse_transform(y_test)


        # Enforce the rule after predictions for childcare expense
        y_train_pred_original[:, childcare_exp_col] = np.where(
            X_train[:, num_children_col] == 0, 0, y_train_pred_original[:, childcare_exp_col])

        y_val_pred_original[:, childcare_exp_col] = np.where(
            X_val[:, num_children_col] == 0, 0, y_val_pred_original[:, childcare_exp_col])

        y_test_pred_original[:, childcare_exp_col] = np.where(
            X_test[:, num_children_col] == 0, 0, y_test_pred_original[:, childcare_exp_col])

        # Metrics for each target
        for i, target in enumerate(target_col_list):
            print(f'Model Results for {target.capitalize()}:')
            print('Train Data:', regr_report(y_train_original[:, i], y_train_pred_original[:, i]))
            print('Validation Data:', regr_report(y_val_original[:, i], y_val_pred_original[:, i]))
            print('Test Data:', regr_report(y_test_original[:, i], y_test_pred_original[:, i])) #Print metrics for test set
            print()

        fold += 1
    return models

# Cross-Validation Execution
xgb_models = cross_val_with_evaluation(X_scaled, y_exp_scaled, kf)

Fold 1/5
[0]	train-rmse:0.09576	eval-rmse:0.09357


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:43:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:0.06498	eval-rmse:0.06382
[100]	train-rmse:0.04792	eval-rmse:0.04759
[150]	train-rmse:0.03894	eval-rmse:0.03937
[200]	train-rmse:0.03424	eval-rmse:0.03522
[250]	train-rmse:0.03154	eval-rmse:0.03279
[300]	train-rmse:0.02986	eval-rmse:0.03120
[350]	train-rmse:0.02847	eval-rmse:0.02993
[400]	train-rmse:0.02729	eval-rmse:0.02883
[450]	train-rmse:0.02634	eval-rmse:0.02797
[500]	train-rmse:0.02578	eval-rmse:0.02749
[550]	train-rmse:0.02528	eval-rmse:0.02708
[600]	train-rmse:0.02482	eval-rmse:0.02670
[650]	train-rmse:0.02448	eval-rmse:0.02645
[700]	train-rmse:0.02410	eval-rmse:0.02616
[750]	train-rmse:0.02364	eval-rmse:0.02578
[800]	train-rmse:0.02330	eval-rmse:0.02550
[850]	train-rmse:0.02298	eval-rmse:0.02523
[900]	train-rmse:0.02266	eval-rmse:0.02493
[950]	train-rmse:0.02233	eval-rmse:0.02463
[1000]	train-rmse:0.02201	eval-rmse:0.02431
[1050]	train-rmse:0.02180	eval-rmse:0.02412
[1100]	train-rmse:0.02156	eval-rmse:0.02391
[1150]	train-rmse:0.02134	eval-rmse:0.02374
[1200]	t

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:44:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:0.07255	eval-rmse:0.07138
[100]	train-rmse:0.04744	eval-rmse:0.04623
[150]	train-rmse:0.03361	eval-rmse:0.03239
[200]	train-rmse:0.02653	eval-rmse:0.02541
[250]	train-rmse:0.02301	eval-rmse:0.02213
[300]	train-rmse:0.02126	eval-rmse:0.02061
[350]	train-rmse:0.02031	eval-rmse:0.01991
[400]	train-rmse:0.01966	eval-rmse:0.01947
[450]	train-rmse:0.01921	eval-rmse:0.01918
[500]	train-rmse:0.01879	eval-rmse:0.01891
[550]	train-rmse:0.01844	eval-rmse:0.01869
[600]	train-rmse:0.01817	eval-rmse:0.01853
[650]	train-rmse:0.01786	eval-rmse:0.01831
[700]	train-rmse:0.01754	eval-rmse:0.01812
[750]	train-rmse:0.01726	eval-rmse:0.01797
[800]	train-rmse:0.01705	eval-rmse:0.01786
[850]	train-rmse:0.01683	eval-rmse:0.01773
[900]	train-rmse:0.01661	eval-rmse:0.01763
[950]	train-rmse:0.01643	eval-rmse:0.01754
[1000]	train-rmse:0.01623	eval-rmse:0.01744
[1050]	train-rmse:0.01606	eval-rmse:0.01735
[1100]	train-rmse:0.01590	eval-rmse:0.01728
[1150]	train-rmse:0.01575	eval-rmse:0.01723
[1200]	t

...

[0]	train-rmse:0.11557	eval-rmse:0.11126
[50]	train-rmse:0.07390	eval-rmse:0.07119
[100]	train-rmse:0.05002	eval-rmse:0.04832
[150]	train-rmse:0.03700	eval-rmse:0.03606
[200]	train-rmse:0.03015	eval-rmse:0.02980
[250]	train-rmse:0.02652	eval-rmse:0.02660
[300]	train-rmse:0.02451	eval-rmse:0.02487
[350]	train-rmse:0.02312	eval-rmse:0.02360
[400]	train-rmse:0.02214	eval-rmse:0.02274
[450]	train-rmse:0.02140	eval-rmse:0.02212
[500]	train-rmse:0.02071	eval-rmse:0.02154
[550]	train-rmse:0.01994	eval-rmse:0.02087
[600]	train-rmse:0.01943	eval-rmse:0.02043
[650]	train-rmse:0.01902	eval-rmse:0.02009
[700]	train-rmse:0.01870	eval-rmse:0.01982
[750]	train-rmse:0.01843	eval-rmse:0.01960
[800]	train-rmse:0.01818	eval-rmse:0.01941
[850]	train-rmse:0.01799	eval-rmse:0.01927
[900]	train-rmse:0.01777	eval-rmse:0.01911
[950]	train-rmse:0.01754	eval-rmse:0.01895
[1000]	train-rmse:0.01735	eval-rmse:0.01882
[1050]	train-rmse:0.01720	eval-rmse:0.01872
[1100]	train-rmse:0.01703	eval-rmse:0.01861
[1150]	trai

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:59:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:0.09960	eval-rmse:0.09560
[100]	train-rmse:0.06871	eval-rmse:0.06631
[150]	train-rmse:0.05199	eval-rmse:0.05081
[200]	train-rmse:0.04292	eval-rmse:0.04256
[250]	train-rmse:0.03791	eval-rmse:0.03808
[300]	train-rmse:0.03486	eval-rmse:0.03535
[350]	train-rmse:0.03277	eval-rmse:0.03356
[400]	train-rmse:0.03141	eval-rmse:0.03243
[450]	train-rmse:0.03053	eval-rmse:0.03168
[500]	train-rmse:0.02982	eval-rmse:0.03106
[550]	train-rmse:0.02907	eval-rmse:0.03044
[600]	train-rmse:0.02864	eval-rmse:0.03007
[650]	train-rmse:0.02821	eval-rmse:0.02971
[700]	train-rmse:0.02777	eval-rmse:0.02935
[750]	train-rmse:0.02737	eval-rmse:0.02905
[800]	train-rmse:0.02702	eval-rmse:0.02879
[850]	train-rmse:0.02672	eval-rmse:0.02855
[900]	train-rmse:0.02645	eval-rmse:0.02835
[950]	train-rmse:0.02624	eval-rmse:0.02820
[1000]	train-rmse:0.02604	eval-rmse:0.02808
[1050]	train-rmse:0.02579	eval-rmse:0.02791
[1100]	train-rmse:0.02555	eval-rmse:0.02776
[1150]	train-rmse:0.02534	eval-rmse:0.02765
[1200]	t

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [21:00:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.09399	eval-rmse:0.08839
[50]	train-rmse:0.05980	eval-rmse:0.05650
[100]	train-rmse:0.03976	eval-rmse:0.03803
[150]	train-rmse:0.02828	eval-rmse:0.02761
[200]	train-rmse:0.02175	eval-rmse:0.02173
[250]	train-rmse:0.01796	eval-rmse:0.01829
[300]	train-rmse:0.01562	eval-rmse:0.01617
[350]	train-rmse:0.01393	eval-rmse:0.01454
[400]	train-rmse:0.01271	eval-rmse:0.01339
[450]	train-rmse:0.01165	eval-rmse:0.01230
[500]	train-rmse:0.01075	eval-rmse:0.01138
[550]	train-rmse:0.00992	eval-rmse:0.01050
[600]	train-rmse:0.00925	eval-rmse:0.00979
[650]	train-rmse:0.00871	eval-rmse:0.00923
[700]	train-rmse:0.00816	eval-rmse:0.00863
[750]	train-rmse:0.00782	eval-rmse:0.00826
[800]	train-rmse:0.00752	eval-rmse:0.00795
[850]	train-rmse:0.00731	eval-rmse:0.00772
[900]	train-rmse:0.00710	eval-rmse:0.00751
[950]	train-rmse:0.00693	eval-rmse:0.00734
[1000]	train-rmse:0.00680	eval-rmse:0.00721
[1050]	train-rmse:0.00670	eval-rmse:0.00712
[1100]	train-rmse:0.00659	eval-rmse:0.00702
[1150]	trai

In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load the scalers
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')

def get_prediction(model_combine, input_data, scaler_X, scaler_y, num_children_col, childcare_exp_col):
    """
    Generates predictions for expenses using the combined model.

    Args:
        model_combine: The trained Keras model.
        input_data: Pandas DataFrame of new input data.
        scaler_X: Fitted MinMaxScaler for features.
        scaler_y: Fitted MinMaxScaler for target variables.
        num_children_col: Integer column index for number of children.
        childcare_exp_col: Integer column index for the 'childcare' target.

    Returns:
         Numpy Array: Predicted expense values.
    """

    # Step 1: Create new features
    input_data_with_features = createFeatures(input_data.copy())

    # Step 2: Select relevant features
    X = input_data_with_features[['total', 'median_family_income', 'num_counties_in_st', 'n_children',
                                  'n_parents', 'n_members', 'per_member_cost', 'child_expense_cost',
                                  'parent_expense_cost', 'other_expense_cost','zero_childcare_cost']].values

    # Step 3: Scale the input data using the fitted scaler_X
    input_data_scaled = scaler_X.transform(X)

    # Step 4: Reshape for the CNN layer (add the extra dimension)
    input_data_scaled = input_data_scaled[..., np.newaxis]  # This line is crucial to match the training data shape

    # Step 5: Make predictions using the trained model
    # We need to unpack the predictions from the model output
    # Then, we can stack those into a single output array.
    input_data_pred = model_combine.predict(input_data_scaled)
    input_data_pred = np.column_stack(input_data_pred)

    # Step 6: Inverse transform the predictions to return them to the original scale
    input_data_pred_original = scaler_y.inverse_transform(input_data_pred)

    # Step 7: Enforce the rule that if there are no children, childcare cost is 0
    input_data_pred_original[:, childcare_exp_col] = np.where(
        input_data_with_features['n_children'].values == 0, 0, input_data_pred_original[:, childcare_exp_col]
    )

    return input_data_pred_original

def createFeatures(input_data):
    """
    Creates additional features needed for the model prediction.

    Args:
        input_data: The input pandas DataFrame with the original data.

    Returns:
        input_data: The input data with additional features.
    """
    # Step 1: Create additional features
    input_data["per_member_cost"] = input_data["total"] / input_data["n_members"]
    input_data["child_expense_cost"] = input_data["per_member_cost"] * input_data["n_children"]
    input_data["parent_expense_cost"] = input_data["per_member_cost"] * input_data["n_parents"]
    input_data["other_expense_cost"] = input_data["total"] - (input_data["child_expense_cost"] + input_data["parent_expense_cost"])

    # Step 2: Create binary column for whether there are children (0 if no children, 1 if there are children)
    input_data["zero_childcare_cost"] = input_data['n_children'].map(lambda x: 0 if x < 1 else 1)

    return input_data

# Example of input data to test predictions
input_data = pd.DataFrame(
    [[4541, 8790.0, 67, 0, 2, 2]],
    columns=['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']
)

# Assuming `model_combine` is your trained model
input_data_pred_original = get_prediction(model_combine, input_data, scaler_X, scaler_y, num_children_col=3, childcare_exp_col=5)

# Print predictions
target_col_list = ['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']
for i, prediction in enumerate(input_data_pred_original[0]):
    print(f"{target_col_list[i]}: {prediction:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
housing: 677.50
food: 605.33
transportation: 1355.80
healthcare: 854.20
othernecessities: 450.93
childcare: 0.00
taxes: 614.43
